In [5]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM 
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np

In [11]:
label_list = []
label_list.append(torch.load("label0.txt")) 
label_list.append(torch.load("label1.txt")) 
#torch.load("file0.txt")...
#save into a list
labels = torch.cat(label_list)

In [12]:
vector_list = []
vector_list.append(torch.load("vector0.txt")) 
vector_list.append(torch.load("vector1.txt")) 
#torch.load("file0.txt")...
#save into a list
final_pool = torch.cat(vector_list)

In [13]:
import torch
from sklearn.model_selection import train_test_split
# convert our data into a torch dataset
#need y data from the file
ratings = torch.tensor(labels).float()

#x_train, x_test, y_train, y_test = train_test_split(pooled.detach(), ratings, test_size=0.15)
x_train, x_test, y_train, y_test = train_test_split(final_pool, ratings[:48], test_size=0.15)

data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_59958/768591445.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ratings = torch.tensor(labels).float()
/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_59958/768591445.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())


In [14]:
model = torch.nn.Sequential(   
    torch.nn.Linear(768,800),
    torch.nn.Sigmoid(),
    torch.nn.Linear(800,1),
    torch.nn.Sigmoid()
)

In [15]:
scoring_function = torch.nn.MSELoss()
num_epochs = 10

optimizer = torch.optim.SGD( model.parameters(), .001 )

In [18]:
from sklearn.metrics import accuracy_score

In [19]:
def train():
    for step in range (num_epochs):

        for batch in loader:
            optimizer.zero_grad()
            batchX = batch[0]
            #print(batchX.size())
            batchY = batch[1].unsqueeze(-1)
            #print(batchY.size())
            # do a gradient descent update of out linear model.

            # put data through the model
            preds = model(batchX)
            #print (preds.shape)
            #fixedPreds=preds[:,0]
            #print("preds: ", fixedPreds)
            # collect the error
            errors = scoring_function(preds, batchY)
            #print("errors: ", errors)

            # compute the gradient of the error with regard to the model paramenters
            errors.backward()

            # take a step in that direction
            optimizer.step()
    new_x_tensor= torch.tensor(x_test).float()
    preds= model(new_x_tensor)
    accuracy= accuracy_score(np.round(preds.detach().numpy()), y_test)
    print(accuracy)
        
            

In [20]:
train()

0.2


/var/folders/dg/jkq_f9b51f38pf3ddscgfrtm0000gn/T/ipykernel_59958/4078027073.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_x_tensor= torch.tensor(x_test).float()
